In [4]:
!ls -a

.		   .gitignore		   behrtmlmazure.ipynb.amltmp
..		   .ipynb_aml_checkpoints  common
.DS_Store	   .vscode		   data
.amlignore	   __pycache__		   dataLoader
.amlignore.amltmp  behrt.yml		   model
.git		   behrtMLMAzure.ipynb	   task


In [3]:
# Move files to datastore and make smaller folder that has only the necessary files

from azureml.core import Workspace, Experiment, Environment
from azureml.core.script_run_config import ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
import azureml._restclient.snapshots_client

# Set the desired snapshot size (in bytes)
snapshot_size = 1073741824

# Update the maximum snapshot size
azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = snapshot_size


# Retrieve Workspace
ws = Workspace.from_config()

# Get the Curated Environment
# env = Environment.get(workspace=ws, name="AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu")
env = Environment.get(workspace=ws, name="AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu")



# # Define additional conda and pip packages you want to install
conda_dep = CondaDependencies()

conda_dep.add_pip_package("lightning")  

env.python.conda_dependencies=conda_dep

# Define the Training Script and Directory
script_folder = ''
script_name = 'task/executeBehrtMLM.py'  # your training script

# Create an Experiment
experiment_name = 'My-Experiment'
experiment = Experiment(workspace=ws, name=experiment_name)

# Configure and Submit the Training Job
src = ScriptRunConfig(source_directory=script_folder,
                      script=script_name,
                      environment=env)  # specifying the cloned environment here

run = experiment.submit(src)
run.wait_for_completion(show_output=True)

ExperimentExecutionException: ExperimentExecutionException:
	Message: {
    "error_details": {
        "componentName": "execution",
        "correlation": {
            "operation": "b8e5051c8d1be3341ef29989e66b550d",
            "request": "4a49f74d45b3d465"
        },
        "environment": "westeurope",
        "error": {
            "code": "UserError",
            "message": "Environment name cannot start with the prefix AzureML. To alter a curated environment first create a copy of it."
        },
        "location": "westeurope",
        "time": "2023-08-12T09:29:10.3240875+00:00"
    },
    "status_code": 400,
    "url": "https://cd7411a6-1444-4dc7-ad9f-0eed2fcbd115.workspace.westeurope.api.azureml.ms/execution/v1.0/subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourceGroups/forskerpl-p2ss0q-rg/providers/Microsoft.MachineLearningServices/workspaces/forskerpl-p2ss0q-mlw/experiments/My-Experiment/localrun?runId=My-Experiment_1691832492_f5a70200"
}
	InnerException None
	Err

In [11]:
!conda env list
!conda activate --stack azureml_py38_PT_TF

# !pip install torchtext

# conda environments:
#
base                     /anaconda
azureml_py310_sdkv2      /anaconda/envs/azureml_py310_sdkv2
azureml_py38             /anaconda/envs/azureml_py38
azureml_py38_PT_TF       /anaconda/envs/azureml_py38_PT_TF
behrt_cpu                /anaconda/envs/behrt_cpu
jupyter_env              /anaconda/envs/jupyter_env



In [8]:

import sys

# sys.path.insert(0, "../")
from common.common import create_folder
from dataLoader.build_vocab import build_vocab
import pytorch_pretrained_bert as Bert
from dataLoader.dataLoaderMLM import MaskedDataset
from model.behrt import BertModel, BertMLM
from torch.utils.data import DataLoader
import json
import os
import lightning.pytorch as pl
from lightning.pytorch.loggers import NeptuneLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.tuner import Tuner

Azure = True

# Initialize Neptune
name_experiment = "MLM_model"
neptune_logger = NeptuneLogger(
    project="sinkjaer/BEHRT",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzOWVmOWI3Mi1jNjliLTQ3NmEtODVjMy0wZjkxZTBiMzFiMzEifQ==",
    log_model_checkpoints=True,
    name=name_experiment,
)


class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get("vocab_size"),
            hidden_size=config["hidden_size"],
            num_hidden_layers=config.get("num_hidden_layers"),
            num_attention_heads=config.get("num_attention_heads"),
            intermediate_size=config.get("intermediate_size"),
            hidden_act=config.get("hidden_act"),
            hidden_dropout_prob=config.get("hidden_dropout_prob"),
            attention_probs_dropout_prob=config.get("attention_probs_dropout_prob"),
            max_position_embeddings=config.get("max_position_embedding"),
            initializer_range=config.get("initializer_range"),
        )
        self.seg_vocab_size = config.get("seg_vocab_size")
        self.age_vocab_size = config.get("age_vocab_size")
        self.date_vocab_size = config.get("date_vocab_size")
        self.optim_param = config.get("optim_param")


if Azure:
    os.environ['NEPTUNE_MODE'] = 'offline'
    file_config = {
        "data_train": "../../EHR_data/data/pre_train_training_set.json",  # formated data
        "data_val": "../../EHR_data/data/pre_train_validation_set.json",  # formated data
        "model_path": "MLM/" + name_experiment,  # where to save model
        "model_name": "behrt",  # model name
        "vocab": "vocab.txt",  # vocabulary idx2token, token2idx
        "file_name": "log.txt",  # log path
    }
else:
    file_config = {
        "data_train": "/Users/mikkelsinkjaer/data/data.json",
        "data_val": "/Users/mikkelsinkjaer/data/data.json",
        "model_path": "MLM/" + name_experiment,  # where to save model
        "model_name": "behrt",  # model name
        "vocab": "vocab.txt",  # vocabulary idx2token, token2idx
        "file_name": "log.txt",  # log path
    }

create_folder(file_config["model_path"])

global_params = {"max_seq_len": 512, "gradient_accumulation_steps": 1}

optim_param = {"lr": 2e-5, "warmup_proportion": 0.1, "weight_decay": 0.01}

train_params = {
    "batch_size": 128,
    "max_len_seq": global_params["max_seq_len"],
}

# load data
with open(file_config["data_train"]) as f:
    data_train_json = json.load(f)
with open(file_config["data_val"]) as f:
    data_val_json = json.load(f)

# Build vocab
vocab_path = os.path.join(file_config["model_path"], file_config["vocab"])
vocab_list, word_to_idx = build_vocab(
    data_train_json,
    save_file=vocab_path,
)

# Data loader
masked_data_train = MaskedDataset(data_train_json, vocab_list, word_to_idx)
trainload = DataLoader(
    dataset=masked_data_train,
    batch_size=train_params["batch_size"],
    shuffle=True,
    pin_memory=True,
    num_workers=6,
)
masked_data_val = MaskedDataset(data_val_json, vocab_list, word_to_idx)
valload = DataLoader(
    dataset=masked_data_val,
    batch_size=train_params["batch_size"],
    shuffle=False,
    pin_memory=True,
    num_workers=6,
)

# Model config
model_config = {
    "vocab_size": len(vocab_list),  # number of disease + symbols for word embedding
    "hidden_size": 288,  # word embedding and seg embedding hidden size
    "seg_vocab_size": 2,  # number of vocab for seg embedding
    "date_vocab_size": int(
        365.25 * 23
    ),  # number of vocab for dates embedding --> days in 23 years
    "age_vocab_size": 144,  # number of vocab for age embedding
    "max_position_embedding": train_params["max_len_seq"],  # maximum number of tokens
    "hidden_dropout_prob": 0.1,  # dropout rate
    "num_hidden_layers": 6,  # number of multi-head attention layers required
    "num_attention_heads": 12,  # number of attention heads
    "attention_probs_dropout_prob": 0.1,  # multi-head attention dropout rate
    "intermediate_size": 512,  # the size of the "intermediate" layer in the transformer encoder
    "hidden_act": "gelu",  # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    "initializer_range": 0.02,  # parameter weight initializer range
    "optim_param": optim_param,  # learning rate
}

# Checkopoint
checkpoint_callback = ModelCheckpoint(
    monitor="metrics/epoch/loss_val",
    dirpath=file_config["model_path"] + "/checkpoints",
    filename="checkpoint-{epoch:02d}",
)

# Define model
neptune_logger.log_hyperparams(model_config)
model = BertModel(BertConfig(model_config))
task = BertMLM(model, BertConfig(model_config))

# Initialize the Trainer with the callback and Neptune logger
trainer = pl.Trainer(
    accelerator = 'gpu',
    logger=neptune_logger,
    max_epochs=10,
    log_every_n_steps=100,
    callbacks=checkpoint_callback,
)


# Train the model as usual
trainer.fit(model=task, train_dataloaders=trainload, val_dataloaders=valload)

ModuleNotFoundError: No module named 'torchtext'